# Group 5 - Gurusankar Gopalakrishnan, Maneel Reddy, Sentiment Analysis of 10K Financial Reports

In [0]:
import os
import pandas as pd
import numpy as np
import pyspark
import datetime

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset
from torch.utils.data import DataLoader


from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

import warnings
import json
import gc
warnings.filterwarnings('ignore')

##### init pytorch GPU optimizations

Because our dataset was really huge, there were some pytorch optimizations that had to be done to ensure the model does not crash.

1. !export PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.5,max_split_size_mb:64 

Explicitly set PYTORCH to automatically garbage collect if resource utlization goes above a threshold, and ensure fragmentation beyong certain memory chunk sizes.

2. Explicitly calling gc.collect() and torch.cuda.empty_cache() to ensure cache and python garbage collection frees up memory.

3. Ensuring all the models are run on GPU, we use .to('cuda') wherever possible, including storing the tokenized data tensors on cuda and the model as well.

In [0]:
!export PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.5,max_split_size_mb:64

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.dropdown('USE_GPU', 'no', ['no', 'yes'])
USE_GPU = dbutils.widgets.get('USE_GPU') == 'yes'

### Initiate Spark session and test Mongo connection

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
# REF_STRING = "mongodb+srv://admin:<password>@msds697-cluster.qzgwq.mongodb.net/"
def read_df_from_mongo(spark, collection_name):
    mongo_username = 'admin'
    mongo_password =  'msds697'
    mongo_ip_address = 'msds697-cluster.qzgwq.mongodb.net/'
    MONGO_DB_NAME = "msds697_project"
    connection_string = f"mongodb+srv://{mongo_username}:{mongo_password}@{mongo_ip_address}{MONGO_DB_NAME}.{collection_name}"
    spark_df = spark.read.format("mongo").option("uri", connection_string).load()
    return spark_df
    
def store_df_to_mongo(spark, spark_df, collection_name):
    mongo_username = 'admin'
    mongo_password =  'msds697'
    mongo_ip_address = 'msds697-cluster.qzgwq.mongodb.net/'
    MONGO_DB_NAME = "msds697_project"
    connection_string = f"mongodb+srv://{mongo_username}:{mongo_password}@{mongo_ip_address}{MONGO_DB_NAME}.{collection_name}"
    spark_df.write.format("com.mongodb.spark.sql.DefaultSource")\
                     .mode("append")\
                     .option("uri", connection_string)\
                     .save()

In [0]:
df = read_df_from_mongo(spark, "financial_7")
df.show(10)

+--------------------+-------+--------------------+-------------------------+--------------------+-----------+--------------------+-----------+---------------+--------------------+--------------------+--------------------+--------------------+----------------+----+--------------+------------+
|                 _id|    cik|             company|complete_text_filing_link|            filename|filing_date|   filing_html_index|filing_type|fiscal_year_end|     htm_filing_link|              item_7|             item_7A|              item_8|period_of_report| sic|state_location|state_of_inc|
+--------------------+-------+--------------------+-------------------------+--------------------+-----------+--------------------+-----------+---------------+--------------------+--------------------+--------------------+--------------------+----------------+----+--------------+------------+
|{64042ad27f6b3d27...| 811589|  FIRST BANCORP /NC/|     https://www.sec.g...|811589_10K_2021_0...| 2022-03-01|https://

<b>ML Goals (Group 5)<b><br>
1. Obtain sentiment scores from Form 10k filings of all companies in the Rusell 3000 Index. The Rusell 3000 Index is a broad market index which represents 98% of the public equity market.<br>

<b>ML Outcome<b><br>
1. Used Finbert to predict sentiment scores(Positive negative and nuetral) for 1960 form 10k filings(~1600 unique stocks). While this is a drop over the initally intended 3000 stocks, the market capitalization for these 1600 stocks should be covering atleast 90% of the public US equity market. The remaining stocks could not be scrapped from Edgar because of the following reasons:-<br>
  (i) Lack of information on Edgar API/delisting of the stock.<br>
  (ii) Change in Ticker information<br>
  (iii) Form 10k information is not in a form that is scrapable or has excessive information which can crash the GPU(Eg. American Insurance Group). The total number of stocks in this group is ~8-10 <br>

Time efficiency (seconds to run) - with the cluster specification, including Databricks Runtime Version, Worker/Driver types, and the number of workers.

Number of CPU cores: 4
Number of GPU cores: 1
PyTorch version: 1.13.1+cu117
Databricks runtime : 12.1 ML (includes Apache Spark 3.3.1, GPU, Scala 2.12)

Worker/Driver type - g4dn.xlarge, 16GB memory, 1 GPU
Number of workers - 5

#### Initiate huggingface finbert pre-trained model

In [0]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to("cuda").half()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [0]:
import torch
import multiprocessing

print("Number of CPU cores:", multiprocessing.cpu_count())
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")


Number of CPU cores: 4
PyTorch version: 1.13.1+cu117
CUDA is available!


In [0]:
torch.cuda.device_count()

Out[8]: 1

In [0]:
torch.cuda.get_device_name()

Out[9]: 'Tesla T4'

### Methods to create sentiment scores [positive, neutral, negative] for sentences raw text using huggingface/pytorch.

In [0]:
def sentimentanalyzer(text = None):
    text = text.split('\n')
    text = [each[:512] for each in text]
    tokenized_text = tokenizer(text, return_tensors='pt',padding = True)['input_ids']
    #import pdb;pdb.set_trace()
    tokenized_text = tokenized_text.to("cuda")
    m = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        model_output = model(tokenized_text)['logits']
    out = m(model_output)
    gc.collect()
    torch.cuda.empty_cache()
    return out

def analysisofoutput(output):
    argmax = torch.argmax(output,dim = 1)
    argmax = argmax.detach().numpy()
    dictionary = {0:'positive',1:'negative',2:'nuetral'}
    return dictionary[argmax[0]]

##### Test sentiment outputs for one record

In [0]:
sentimentanalyzer(df.select("item_7").limit(1).collect()[0]["item_7"]).cpu().numpy().tolist()

Out[24]: [[0.09722900390625, 0.209716796875, 0.693359375],
 [0.0291900634765625, 0.1614990234375, 0.80908203125],
 [0.1328125, 0.1358642578125, 0.7314453125],
 [0.179443359375, 0.01271820068359375, 0.80810546875],
 [0.06268310546875, 0.0106658935546875, 0.9267578125],
 [0.8798828125, 0.01450347900390625, 0.10565185546875],
 [0.08111572265625, 0.1805419921875, 0.73828125],
 [0.7412109375, 0.171630859375, 0.08709716796875],
 [0.9306640625, 0.0151824951171875, 0.053955078125],
 [0.609375, 0.279052734375, 0.1116943359375],
 [0.044677734375, 0.7197265625, 0.235595703125],
 [0.0214996337890625, 0.9599609375, 0.0185546875],
 [0.072998046875, 0.45263671875, 0.474365234375],
 [0.330078125, 0.1943359375, 0.4755859375],
 [0.88916015625, 0.01068878173828125, 0.10028076171875],
 [0.67529296875, 0.02044677734375, 0.304443359375],
 [0.08807373046875, 0.167724609375, 0.744140625],
 [0.1510009765625, 0.2020263671875, 0.64697265625],
 [0.2144775390625, 0.345458984375, 0.440185546875],
 [0.86572265625, 0

#### Create sentiment scores vectors for spark_df

In [0]:
#Create UDF to add sentiment analysis 
sentiment_vectors_func = udf(lambda x: sentimentanalyzer(x).cpu().numpy().tolist(), ArrayType(ArrayType(FloatType())))

def sentiment_vectorizer(spark_df, large = False): 
    sentiment_df = spark_df.withColumn("vectors", sentiment_vectors_func(col("item_7")))
    
    drop_list = ['item_7A','item_8','item_7']
    sentiment_df = sentiment_df.drop(*drop_list)
    
    return sentiment_df


####test for few records

In [0]:
#test for few records
sentiment_vectorizer(df.sample(0.01)).show()


+--------------------+-------+--------------------+-------------------------+--------------------+-----------+--------------------+-----------+---------------+--------------------+----------------+----+--------------+------------+------+--------------------+
|                 _id|    cik|             company|complete_text_filing_link|            filename|filing_date|   filing_html_index|filing_type|fiscal_year_end|     htm_filing_link|period_of_report| sic|state_location|state_of_inc|length|             vectors|
+--------------------+-------+--------------------+-------------------------+--------------------+-----------+--------------------+-----------+---------------+--------------------+----------------+----+--------------+------------+------+--------------------+
|{64042af77f6b3d27...|1856653|BioPlus Acquisiti...|     https://www.sec.g...|1856653_10K_2021_...| 2022-03-11|https://www.sec.g...|       10-K|           1231|https://www.sec.g...|      2021-12-31|6770|            NY|      

#### create sentiment score vectors for all records and store to mongo for later use

In [0]:
df = df.withColumn("length",length("item_7")).sort("length")
df.count()

Out[55]: 1971

In [0]:
sentiment_df = sentiment_vectorizer(df)

In [0]:
# sentiment_df = sentiment_vectorizer(df)
store_df_to_mongo(spark, sentiment_df, "sentiment_raw_financial_vectors_2")

---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
File <command-3451456928319836>:2
      1 # sentiment_df = sentiment_vectorizer(df)
----> 2 store_df_to_mongo(spark, sentiment_df, "sentiment_raw_financial_vectors_2")

File <command-3451456928319667>:17, in store_df_to_mongo(spark, spark_df, collection_name)
     15 MONGO_DB_NAME = "msds697_project"
     16 connection_string = f"mongodb+srv://{mongo_username}:{mongo_password}@{mongo_ip_address}{MONGO_DB_NAME}.{collection_name}"
---> 17 spark_df.write.format("com.mongodb.spark.sql.DefaultSource")\
     18                  .mode("append")\
     19                  .option("uri", connection_string)\
     20                  .save()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
 

In [0]:
sentiment_df.count()

Out[63]: 1971

##### AVOID CUDA OVERFLOW CHECK #####

In [0]:
sentiment_df = read_df_from_mongo(spark, "sentiment_raw_financial_vectors_2")

In [0]:
sentiment_df.count()

Out[65]: 1962

In [0]:
senti_feats_df = sentiment_df.select('company','vectors','state_of_inc','state_location')
senti_feats_df.printSchema()

root
 |-- company: string (nullable = true)
 |-- vectors: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- state_of_inc: string (nullable = true)
 |-- state_location: string (nullable = true)



In [0]:
senti_feats_df.take(5)

Out[67]: [Row(company='Pacira BioSciences, Inc.', vectors=[[0.08782958984375, 0.1942138671875, 0.7177734375], [0.029998779296875, 0.04119873046875, 0.9287109375], [0.0239410400390625, 0.0670166015625, 0.9091796875], [0.10321044921875, 0.08929443359375, 0.8076171875], [0.263671875, 0.007717132568359375, 0.728515625], [0.62841796875, 0.0081787109375, 0.363525390625], [0.115234375, 0.09747314453125, 0.787109375], [0.3662109375, 0.007587432861328125, 0.6259765625], [0.075927734375, 0.07562255859375, 0.8486328125], [0.1339111328125, 0.31640625, 0.5498046875], [0.11865234375, 0.166748046875, 0.71484375], [0.0830078125, 0.014434814453125, 0.90234375], [0.11785888671875, 0.1007080078125, 0.78125], [0.037994384765625, 0.051055908203125, 0.9111328125], [0.0310821533203125, 0.0217437744140625, 0.947265625], [0.13037109375, 0.169921875, 0.69970703125], [0.03228759765625, 0.70703125, 0.260498046875], [0.03265380859375, 0.050140380859375, 0.9169921875], [0.1314697265625, 0.314208984375, 0.5541992187

In [0]:
senti_flat_scores = senti_feats_df.rdd.map(lambda x: x["vectors"]).map(lambda x: np.array(x))\
.map(lambda x: x[x[:,2]<0.5].sum(axis = 0))\
.map(lambda x: x/x.sum())

senti_flat_scores.count()

Out[68]: 1962

In [0]:
schema = StructType([
    StructField("pos_sentiment_score", DoubleType(), True),
    StructField("neg_sentiment_score", DoubleType(), True),
    StructField("neutral_sentiment_score", DoubleType(), True),
])

def float_safe(f):
    try:
        return float(f)
    except ValueError:
        return None

scores_df = spark.createDataFrame(senti_flat_scores.map(lambda x: (float_safe(x[0]),float_safe(x[1]),float_safe(x[2]))), schema)
senti_feats_df = senti_feats_df.withColumn("uid", monotonically_increasing_id())
scores_df = scores_df.withColumn("uid", monotonically_increasing_id())

merged_df = senti_feats_df.join(scores_df, "uid")
clus_feats = merged_df.drop("vectors")

In [0]:
clus_feats.printSchema()
# clus_feats.show(5)

store_df_to_mongo(spark, clus_feats, "sentiment_feats_1962")

root
 |-- uid: long (nullable = false)
 |-- company: string (nullable = true)
 |-- state_of_inc: string (nullable = true)
 |-- state_location: string (nullable = true)
 |-- pos_sentiment_score: double (nullable = true)
 |-- neg_sentiment_score: double (nullable = true)
 |-- neutral_sentiment_score: double (nullable = true)



In [0]:
cleaned_sentiment_scores = read_df_from_mongo(spark, "sentiment_feats_1962")

In [0]:
cleaned_sentiment_scores.show()

+--------------------+--------------------+-------------------+-----------------------+-------------------+--------------+------------+----+
|                 _id|             company|neg_sentiment_score|neutral_sentiment_score|pos_sentiment_score|state_location|state_of_inc| uid|
+--------------------+--------------------+-------------------+-----------------------+-------------------+--------------+------------+----+
|{640bb0d34412ca47...|      NEWMARKET CORP| 0.4827770467229484|     0.2136565382343032| 0.3035664150427484|            VA|          VA|  26|
|{640bb0d34412ca47...|CROSS COUNTRY HEA...|0.20764179080185705|    0.12475027159870934| 0.6676079375994336|            FL|          DE|  29|
|{640bb0d34412ca47...| WILLIAMS SONOMA INC|0.37441371817864677|    0.20228328289843744| 0.4233029989229158|            CA|          CA| 474|
|{640bb0d34412ca47...|REGENERON PHARMAC...|  0.406919183552471|    0.23853450981759844| 0.3545463066299306|            NY|          NY| 964|
|{640bb0d3441